In [40]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
sexes = df.sex_upon_outcome.unique()


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


image_filename = 'logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Database Enhancement'))),
    html.Hr(),
    #html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div(
    dcc.RadioItems(
        id = 'radioItem',
        options=[
            {'label' : 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness', 'value' : 'MoW'},
            {'label': 'Disaster or Individual Tracking', 'value' : "DoIT"},
            {'label' : 'All Items', 'value' : 'all'}
        ],
        value = 'all',
        labelStyle = {'display' : 'inline-block'}
    ),
    ),
    html.Hr(),
    
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,  
    ),
    html.Br(),
    html.Hr(),
    # create a div for the map data
    html.Div(className='row',
         style={'display' : 'flex'},
         children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',

                ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                )
        ]),
    html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} for x in sexes],
        value=sexes[0],
        clearable=False,
    ),
    dcc.Graph(id="bar-chart"),
])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output("bar-chart", "figure"), 
    [Input("dropdown", "value")])
def update_bar_chart(sex):
    mask = df["sex_upon_outcome"] == sex
    fig = px.bar(df[mask], x="age_upon_outcome", y="breed", 
                 color="animal_type", barmode="group")
    return fig                               
    
@app.callback(
    Output('datatable-id', "data"),
    [Input('radioItem', 'value')])

def update_dashboard(value):

    if value == 'all':
        df = pd.DataFrame.from_records(shelter.read({}))
    elif value == 'WR':
        df = pd.DataFrame.from_records(shelter.read({"breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female", "age_upon_outcome_in_weeks":{"$gt":"26"},"age_upon_outcome_in_weeks":{"$lt":"156"}}))
    elif value == 'MoW':
        df = pd.DataFrame.from_records(shelter.read({"breed":{"$in":["German Shepherd,","Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottweiler" ]}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gt":"26"},"age_upon_outcome_in_weeks":{"$lt":"156"}}))
    elif value == 'DoIT':
        df = pd.DataFrame.from_records(shelter.read({"breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gt":"20"},"age_upon_outcome_in_weeks":{"$lt":"300"}}))   
    
    return df.to_dict('records')



@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed')
    return [
        dcc.Graph(            
            figure = fig
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graph2(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.bar(df, x='breed', y='age_upon_outcome')
    fig.show()


app